In [36]:
import findspark
findspark.init()
findspark.find()

'C:\\spark\\spark3'

In [2]:
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

In [3]:
 spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [4]:
from pyspark.sql.functions import when
from pyspark.sql.functions import col,lit

In [5]:
from pyspark.sql.types import StructType,StructField,StringType,LongType,IntegerType

In [6]:
df = spark.read.csv(r"C:\Users\goelp\OneDrive\Desktop\archive\bank.csv",header = True,inferSchema = True)

In [7]:
df.show(5)

+----------+---------+--------------------+------+----------+------------+------------+------------+-------+
|Account_No|     DATE|       TRANS_DETAILS|CHQ_NO|VALUE_DATE|WITHDRAW_AMT| DEPOSIT_AMT| BALANCE_AMT|Tras_id|
+----------+---------+--------------------+------+----------+------------+------------+------------+-------+
|4998365300|29-Jun-17|TRF FROM  Indiafo...|  null| 29-Jun-17|        null|10,00,000.00|10,00,000.00|      1|
|4998365301|05-Jul-17|TRF FROM  Indiafo...|  null| 05-Jul-17|        null|10,00,000.00|20,00,000.00|      2|
|4998365302|18-Jul-17|FDRL/INTERNAL FUN...|  null| 18-Jul-17|        null| 5,00,000.00|25,00,000.00|      3|
|4998365303|01-Aug-17|TRF FRM  Indiafor...|  null| 01-Aug-17|        null|30,00,000.00|55,00,000.00|      4|
|4998365304|16-Aug-17|FDRL/INTERNAL FUN...|  null| 16-Aug-17|        null| 5,00,000.00|60,00,000.00|      5|
+----------+---------+--------------------+------+----------+------------+------------+------------+-------+
only showing top 5 

In [8]:
df.printSchema()

root
 |-- Account_No: long (nullable = true)
 |-- DATE: string (nullable = true)
 |-- TRANS_DETAILS: string (nullable = true)
 |-- CHQ_NO: integer (nullable = true)
 |-- VALUE_DATE: string (nullable = true)
 |-- WITHDRAW_AMT: string (nullable = true)
 |-- DEPOSIT_AMT: string (nullable = true)
 |-- BALANCE_AMT: string (nullable = true)
 |-- Tras_id: integer (nullable = true)



In [9]:
df1 = df.filter(df.Account_No == "4998365300")

In [10]:
df1.show()

+----------+---------+--------------------+------+----------+--------------+------------+------------------+-------+
|Account_No|     DATE|       TRANS_DETAILS|CHQ_NO|VALUE_DATE|  WITHDRAW_AMT| DEPOSIT_AMT|       BALANCE_AMT|Tras_id|
+----------+---------+--------------------+------+----------+--------------+------------+------------------+-------+
|4998365300|29-Jun-17|TRF FROM  Indiafo...|  null| 29-Jun-17|          null|10,00,000.00|      10,00,000.00|      1|
|4998365300|25-Sep-18|Indiaforensic RUP...|  null| 25-Sep-18|         28.22|        null|  -54,37,98,937.36|  10001|
|4998365300|19-Jan-18|AEPS ACQUIRING AD...|  null| 19-Jan-18|         14.75|        null|  -53,63,87,887.14|  20001|
|4998365300|16-Oct-15|INTERNAL FUND TRA...|  null| 16-Oct-15|1,20,00,000.00|        null|-1,45,71,54,949.93|  30001|
|4998365300|28-Jul-15|INTERNAL FUND TRA...|  null| 28-Jul-15|  30,00,000.00|        null|-1,58,65,35,962.60|  40001|
|4998365300|10-May-16|RTGS/YESBH1613189...|  null| 10-May-16|1,5

In [11]:
df2 = df.na.fill("")

In [12]:
df2.show()

+----------+---------+--------------------+------+----------+------------+------------+------------+-------+
|Account_No|     DATE|       TRANS_DETAILS|CHQ_NO|VALUE_DATE|WITHDRAW_AMT| DEPOSIT_AMT| BALANCE_AMT|Tras_id|
+----------+---------+--------------------+------+----------+------------+------------+------------+-------+
|4998365300|29-Jun-17|TRF FROM  Indiafo...|  null| 29-Jun-17|            |10,00,000.00|10,00,000.00|      1|
|4998365301|05-Jul-17|TRF FROM  Indiafo...|  null| 05-Jul-17|            |10,00,000.00|20,00,000.00|      2|
|4998365302|18-Jul-17|FDRL/INTERNAL FUN...|  null| 18-Jul-17|            | 5,00,000.00|25,00,000.00|      3|
|4998365303|01-Aug-17|TRF FRM  Indiafor...|  null| 01-Aug-17|            |30,00,000.00|55,00,000.00|      4|
|4998365304|16-Aug-17|FDRL/INTERNAL FUN...|  null| 16-Aug-17|            | 5,00,000.00|60,00,000.00|      5|
|4998365305|16-Aug-17|FDRL/INTERNAL FUN...|  null| 16-Aug-17|            | 5,00,000.00|65,00,000.00|      6|
|4998365306|16-Aug-

In [13]:
Bank_log_details = df2.select("Account_No",
                              "DATE",
                              "TRANS_DETAILS",
                              "CHQ_NO",
                              "VALUE_DATE",
                              "WITHDRAW_AMT",
                              "DEPOSIT_AMT",
                              "BALANCE_AMT",
                              "Tras_id").drop_duplicates()

In [14]:
Bank_log_details = Bank_log_details.withColumn("Action", \
   when((Bank_log_details.WITHDRAW_AMT !=""), lit("Withdraw"))\
     .otherwise(lit("Deposit")))

In [15]:
Bank_log_details.show()

+----------+---------+--------------------+------+----------+------------+------------+----------------+-------+--------+
|Account_No|     DATE|       TRANS_DETAILS|CHQ_NO|VALUE_DATE|WITHDRAW_AMT| DEPOSIT_AMT|     BALANCE_AMT|Tras_id|  Action|
+----------+---------+--------------------+------+----------+------------+------------+----------------+-------+--------+
|4998365327|16-Aug-17|INDO GIBL Indiafo...|  null| 16-Aug-17|   78,100.00|            |    62,01,467.00|     28|Withdraw|
|4998365561|07-Feb-18|INDO GIBL Indiafo...|  null| 07-Feb-18|   61,500.00|            |    14,35,695.00|    262|Withdraw|
|4998366033|29-Sep-18|Indfor INCOME IND...|  null| 29-Sep-18|    1,920.00|            |    10,28,562.00|    734|Withdraw|
|4998366937|01-Mar-18|FDRL/INTERNAL FUN...|  null| 01-Mar-18|            | 3,00,000.00|    11,04,941.41|   1638| Deposit|
|4998366986|09-Apr-18|FDRL/INTERNAL FUN...|  null| 09-Apr-18|            | 3,00,000.00|     8,80,613.78|   1687| Deposit|
|4998367070|05-Jun-18|  

In [62]:
output_path = r"C:\Users\goelp\OneDrive\Desktop\Output_files\Bank_log_files"

In [64]:
Bank_log_details.write.mode('overwrite').parquet(os.path.join(output_path))

In [37]:
Schem = StructType(
[
  
  StructField("Account_No",LongType(),False),
  StructField("Age",IntegerType(),False),
  StructField("Gender",StringType(),False),
  StructField("Education",StringType(),False),
  StructField("Marital_Status",StringType(),False),
  StructField("Acc_Type",StringType(),False),
  StructField("Months_On_Bank",IntegerType(),False),
  StructField("Credit_Card",StringType(),False),
  StructField("Credit_Score",IntegerType(),False),
  StructField("Estimated_Income",StringType(),False),
  StructField("Cust_id",IntegerType(),False),
  StructField("corrupt_record",StringType(),True)  
    
]

)

In [38]:
df = (
            spark.read
            .option("mode","PERMISSIVE")
            .option("columnNameOfCorruptRecord","corrupt_record")
            .csv(r"C:\Users\goelp\OneDrive\Desktop\archive\Bank Customer Details.csv",header = True,schema = Schem)
     )

In [39]:
df.show(5)

+----------+---+------+-----------+--------------+--------+--------------+-----------+------------+----------------+-------+--------------+
|Account_No|Age|Gender|  Education|Marital_Status|Acc_Type|Months_On_Bank|Credit_Card|Credit_Score|Estimated_Income|Cust_id|corrupt_record|
+----------+---+------+-----------+--------------+--------+--------------+-----------+------------+----------------+-------+--------------+
|4998365300| 45|  Male|High School|       Married| Savings|            42|        Yes|         619|       60K - 80K|      1|          null|
|4998365301| 49|Female|   Graduate|        Single| Savings|            41|         No|         608|   Less than 40K|      2|          null|
|4998365302| 51|  Male|   Graduate|       Married| Savings|            42|        Yes|         502|      80K - 120K|      3|          null|
|4998365303| 40|Female|High School|          null| Savings|            39|         No|         699|   Less than 40K|      4|          null|
|4998365304| 40|  Ma

In [40]:
df.printSchema()

root
 |-- Account_No: long (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Acc_Type: string (nullable = true)
 |-- Months_On_Bank: integer (nullable = true)
 |-- Credit_Card: string (nullable = true)
 |-- Credit_Score: integer (nullable = true)
 |-- Estimated_Income: string (nullable = true)
 |-- Cust_id: integer (nullable = true)
 |-- corrupt_record: string (nullable = true)



In [41]:
df.filter(df.Account_No == "5998375299").show()

+----------+---+------+---------+--------------+--------+--------------+-----------+------------+----------------+-------+--------------------+
|Account_No|Age|Gender|Education|Marital_Status|Acc_Type|Months_On_Bank|Credit_Card|Credit_Score|Estimated_Income|Cust_id|      corrupt_record|
+----------+---+------+---------+--------------+--------+--------------+-----------+------------+----------------+-------+--------------------+
|5998375299| 51|     1| Graduate|       Married| Savings|          null|        Yes|         792|      80K - 120K|  10000|5998375299,51,1,G...|
|5998375299| 51|  Male|        0|       Married| Savings|            28|        Yes|         792|      80K - 120K|  10000|                null|
+----------+---+------+---------+--------------+--------+--------------+-----------+------------+----------------+-------+--------------------+



In [42]:
df.show()

+----------+---+------+-------------+--------------+--------+--------------+-----------+------------+----------------+-------+--------------+
|Account_No|Age|Gender|    Education|Marital_Status|Acc_Type|Months_On_Bank|Credit_Card|Credit_Score|Estimated_Income|Cust_id|corrupt_record|
+----------+---+------+-------------+--------------+--------+--------------+-----------+------------+----------------+-------+--------------+
|4998365300| 45|  Male|  High School|       Married| Savings|            42|        Yes|         619|       60K - 80K|      1|          null|
|4998365301| 49|Female|     Graduate|        Single| Savings|            41|         No|         608|   Less than 40K|      2|          null|
|4998365302| 51|  Male|     Graduate|       Married| Savings|            42|        Yes|         502|      80K - 120K|      3|          null|
|4998365303| 40|Female|  High School|          null| Savings|            39|         No|         699|   Less than 40K|      4|          null|
|49983

In [43]:
Customer = df.select("Cust_id",
                      "Gender",
                      "Education",
                      "Marital_Status",
                       "Age"
                    ).drop_duplicates()

In [44]:
Customer.show()

+-------+------+-------------+--------------+----+
|Cust_id|Gender|    Education|Marital_Status| Age|
+-------+------+-------------+--------------+----+
|     24|Female|      Unknown|        Single|null|
|    261|  Male|      College|       Unknown|  36|
|    462|  Male|      Unknown|        Single|  38|
|    661|  Male|   Uneducated|       Unknown|null|
|   1089|  Male|      College|       Married|  35|
|   1320|  Male|     Graduate|       Married|  48|
|   1562|Female|      College|       Married|  48|
|   2136|  Male|  High School|        Single|  36|
|   2155|  Male|      College|        Single|  26|
|   2358|  Male|      College|       Married|  57|
|   3065|  Male|     Graduate|       Married|  45|
|   3108|Female|     Graduate|       Married|  43|
|   3130|Female|      College|      Divorced|  53|
|   3222|  Male|  High School|       Unknown|  35|
|   3321|Female|      Unknown|       Married|  41|
|   3534|  Male|   Uneducated|       Married|  44|
|   3551|Female|  High School| 

In [45]:
Account = df.select("Account_No",
                    "Acc_Type",
                    "Months_On_Bank",
                    "Credit_Score",
                    "Credit_Card",
                    "Estimated_Income"
                    ).drop_duplicates()

In [48]:
df.filter(col("corrupt_record").isNotNull()).show()

Py4JJavaError: An error occurred while calling o189.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 28.0 failed 1 times, most recent failure: Lost task 0.0 in stage 28.0 (TID 42, LAPTOP-2743R0QM, executor driver): java.lang.IllegalArgumentException: corrupt_record does not exist. Available: Account_No, Age, Gender, Education, Marital_Status, Acc_Type, Months_On_Bank, Credit_Card, Credit_Score, Estimated_Income, Cust_id
	at org.apache.spark.sql.types.StructType.$anonfun$fieldIndex$1(StructType.scala:306)
	at scala.collection.MapLike.getOrElse(MapLike.scala:131)
	at scala.collection.MapLike.getOrElse$(MapLike.scala:129)
	at scala.collection.AbstractMap.getOrElse(Map.scala:63)
	at org.apache.spark.sql.types.StructType.fieldIndex(StructType.scala:305)
	at org.apache.spark.sql.catalyst.csv.CSVFilters.$anonfun$predicates$4(CSVFilters.scala:65)
	at org.apache.spark.sql.catalyst.csv.CSVFilters.$anonfun$predicates$4$adapted(CSVFilters.scal

In [ ]:
Corrupt_records = df.select("corrupt_record")